In [1]:
%matplotlib inline

In [2]:
#import dependencies
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# from sqlalchemy import create_engine
database_path = "../Data/employees.sqlite"

In [4]:
#Create a connection to the sql server/database
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [ ]:
# data = engine.execute("SELECT * FROM employees")
# for record in data:
#     print(record)

In [ ]:
#Query the salaries table
salaries=pd.read_sql("SELECT * FROM salaries",conn)
salaries.head()

In [13]:
#Query the employees table
Employees = pd.read_sql("SELECT * FROM "Employees"", conn)
Employees.head()

SyntaxError: invalid syntax (<ipython-input-13-9f1c8551976a>, line 2)

In [ ]:
#Query the titles table
titles = pd.read_sql("SELECT * FROM titles",conn)
titles.head()

In [ ]:
#Merge the tables together
#First -  join the salaries table to employees table on emp_no
emp_sal = employees.merge(salaries, on = "emp_no")
emp_sal.head()

In [ ]:
#Second - join the titles table to the newly created database (emp_sal) on emp_title_id and title_id
final_db = emp_sal.merge(titles, left_on = "emp_title_id", right_on = "title_id")
final_db.head()

In [ ]:
#Extract a database of only the titles and salaries
sal_title_db = final_db[['salary','title']]
sal_title_db.head()

In [ ]:
#Count the number of rows
sal_title_db.count()

In [ ]:
#Group the Titles using groupby and mean() for the Salaries so we can graph
#Round() to zero digits since they don't add anything to the data
sal_title_db.groupby('title')['salary'].mean().round(0)

In [ ]:
#Create a Histogram of the most common salary ranges for employees
sal_title_db.hist(column='salary',color = 'salmon')
plt.xlabel('Salary Range')
plt.ylabel('Salary Frequency/Count')
plt.title('Salary Average Distribution')

In [ ]:
 # Create a Bar Chart of Average Salary by title
sal_title_db2 = sal_title_db.groupby(['title'])['salary'].mean()
sal_title_db2.plot.barh(color='lightblue')
plt.ylabel('Job Title')
plt.xlabel('Avg Salary')
plt.title('A Look at Salary Averages Across Job Titles')
plt.show()

In [ ]:
emp = final_db.loc[final_db['emp_no'] == 499942]
emp